In [1]:
!pip install bs4
#this module helps to fetch the content of the website from internet
!pip install request
#provides functions to search the html content
!pip install html5lib 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
headers = {
  'User-Agents': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}
url = "https://www.flipkart.com/search?q=mobiles+under+50000+rupees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1"  # Website that we want to scrap
url

'https://www.flipkart.com/search?q=mobiles+under+50000+rupees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1'

In [4]:
r = requests.get(url,headers=headers) # Request to url
r

<Response [200]>

In [5]:
htmlcontent=r.content  # Extract the html content of the webpage

In [6]:
htmlcontent

b'<!doctype html><html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.8dd48d.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.c48a12.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.e4e719.css"/><meta http-equiv="Content-type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=Edge"/><meta property="fb:page_id" content="102988293558"/><meta property="fb:admins" content="658873552,624500995,100000233612389"/><link rel="shortcut icon" href="https:///www/promos/new/20150528-140547-favicon-retina.ico"/><link type="application/opensearchdescription+xml" rel="search" href="/osdd.xml?v=2"/><meta property="og:type" content="website"/><meta name="og_site_name" property="og:site_name" content="Flipk

# Beautiful soup for parsing the html content

In [7]:
soup = BeautifulSoup(htmlcontent , 'html.parser',)
soup

<!DOCTYPE html>
<html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.8dd48d.css" rel="stylesheet"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.c48a12.css" rel="stylesheet"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.e4e719.css" rel="stylesheet"/><meta content="text/html; charset=utf-8" http-equiv="Content-type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="102988293558" property="fb:page_id"/><meta content="658873552,624500995,100000233612389" property="fb:admins"/><link href="https:///www/promos/new/20150528-140547-favicon-retina.ico" rel="shortcut icon"/><link href="/osdd.xml?v=2" rel="search" type="application/opensearchdescription+xml"/><meta content="website" property="og:type"/><meta content="Flipkart.com" name="og_site_name" property="og:sit

In [8]:
product_name = []
prices_list= []
descriptions_list =[]
reviews_list = []

req_area = soup.find('div',class_="DOjaWF gdgoEp")   ## only the required area of the page to get the productnames,prices,descriptions,reviews


names = req_area.find_all('div',class_="KzDlHZ")   ## Product names and its name of the class
prices = req_area.find_all('div',class_="Nx9bqj _4b5DiR")
descriptions = req_area.find_all('ul',class_="G4BRas")
reviews = req_area.find_all('div',class_='XQDdHH')

for i in names:
    name=i.text
    product_name.append(name)
    
for i in prices:
    price = i.text
    # print(price)
    prices_list.append(price)

for i in descriptions:
    description = i.text
    descriptions_list.append(description)

for i in reviews:
    review = i.text
    reviews_list.append(review)

df = pd.DataFrame({
    'Products':product_name,
    'Prices':prices_list,
    'Reviews':reviews_list,
    'Description':descriptions_list
})


In [9]:
df.shape

(24, 4)

## To get the class name and link(href) of anchor tag of the next page

In [10]:
#Since there are multiple webpages , so to got to the next page (np) we get the class name of the 'Next Page'
np = soup.find('a',class_='_9QVEpD').get('href')
np = 'https://www.flipkart.com'+np
np

'https://www.flipkart.com/search?q=mobiles+under+50000+rupees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=2'

In [11]:
# To scrap one at a time

# **********To move from one page to another (Traversing the pages)

In [27]:
for items in range(2,10):
    url = f"https://www.flipkart.com/search?q=mobiles+under+50000+rupees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={items}"
    r = requests.get(url=url,headers=headers)
    htmlresponse = r.content
    soup = BeautifulSoup(htmlresponse,'html.parser')

    req_area = soup.find('div',class_="DOjaWF gdgoEp")   ## only the required area of the page to get the productnames,prices,descriptions,reviews
    if req_area is None:
        print(f"No required area found on page {items}. Skipping...")
        continue
    
    names = req_area.find_all('div',class_="KzDlHZ")   ## Product names and its name of the class
    prices = req_area.find_all('div',class_="Nx9bqj _4b5DiR")
    descriptions = req_area.find_all('ul',class_="G4BRas")
    reviews = req_area.find_all('div',class_='XQDdHH')

    # max_items = max(len(names), len(prices), len(descriptions), len(reviews))    #  Finds the largest number of elements among names, prices, descriptions, and reviews for the current page since there 
    for i in names:
        if i.text:
            product_name.append(i.text)
        else:
            product_name.append('n/a')
        
        
    for i in prices:
        if i.text:
        # print(price)
            prices_list.append(i.text)
        else:
            prices_list.append('n/a')
    
    for i in descriptions:
        if i.text:
            descriptions_list.append(i.text)
        else:
            descriptions_list.append('n/a')
    
    for i in reviews:
        if i.text:
            reviews_list.append(i.text)
        else:
            reviews_list.append('n/a')

    # np = soup.find('a',class_='_9QVEpD')
    # next_page = 'https://www.flipkart.com' + np.get('href')
    # print(next_page)


No required area found on page 4. Skipping...


In [28]:
len(product_name)

336

In [29]:
len(descriptions_list)

336

In [30]:
len(reviews_list)

328

In [31]:
len(prices_list)

333

In [32]:
df = pd.DataFrame({
    'Products':product_name,
    'Prices':prices_list,
    'Reviews':reviews_list,
    'Description':descriptions_list
})

ValueError: All arrays must be of the same length

In [18]:
df.shape

(4632, 4)

In [19]:
df.head(30)

,Products,Prices,Reviews,Description
0,"Apple iPhone 13 (Midnight, 128 GB)","₹45,399",4.6,128 GB ROM15.49 cm (6.1 inch) Super Retina XDR...
1,"Apple iPhone 13 (Blue, 128 GB)","₹45,399",4.6,128 GB ROM15.49 cm (6.1 inch) Super Retina XDR...
2,"Motorola Edge 50 Ultra 5G (Forest Grey, 512 GB)","₹49,999",4.4,12 GB RAM | 512 GB ROM17.02 cm (6.7 inch) Disp...
3,"Google Pixel 8a (Obsidian, 256 GB)","₹46,999",4.2,8 GB RAM | 256 GB ROM15.49 cm (6.1 inch) Full ...
4,"Motorola Edge 50 Ultra 5G (Peach Fuzz, 512 GB)","₹49,999",4.4,12 GB RAM | 512 GB ROM17.02 cm (6.7 inch) Disp...
5,"Motorola Edge 50 Ultra 5G (Nordic Wood, 512 GB)","₹49,999",4.4,12 GB RAM | 512 GB ROM17.02 cm (6.7 inch) Disp...
6,"vivo V40 Pro 5G (Ganges Blue, 256 GB)","₹49,999",4.5,8 GB RAM | 256 GB ROM17.22 cm (6.78 inch) Full...
7,"vivo V40 Pro 5G (Titanium Grey, 256 GB)","₹49,999",4.5,8 GB RAM | 256 GB ROM17.22 cm (6.78 inch) Full...
8,"Infinix Zero Flip 5G (Rock Black, 512 GB)","₹49,999",4.5,8 GB RAM | 512 GB ROM17.53 cm (6.9 inch) Full ...
9,"Infinix Zero Flip 5G (Blossom Glow, 512 GB)","₹49,999",4.5,8 GB RAM | 512 GB ROM17.53 cm (6.9 inch) Full ...


In [20]:
df.isnull().sum()

Products       0
Prices         0
Reviews        0
Description    0
dtype: int64

In [21]:
df.shape

(4632, 4)

In [22]:
product_name = []
prices_list = []
descriptions_list = []
reviews_list = []

for items in range(2, 10):
    url = f"https://www.flipkart.com/search?q=mobiles+under+50000+rupees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={items}"
    r = requests.get(url=url, headers=headers)
    if r.status_code != 200:
        print(f"Failed to fetch page {items}. Status code: {r.status_code}")
        continue
    
    htmlresponse = r.content
    soup = BeautifulSoup(htmlresponse, 'html.parser')

    req_area = soup.find('div', class_="DOjaWF gdgoEp")  # Only the required area of the page

    names = req_area.find_all('div', class_="KzDlHZ")
    prices = req_area.find_all('div', class_="Nx9bqj _4b5DiR")
    descriptions = req_area.find_all('ul', class_="G4BRas")
    reviews = req_area.find_all('div', class_='XQDdHH')

    # Determine the maximum number of items for this page
    max_items = max(len(names), len(prices), len(descriptions), len(reviews))

    for i in range(max_items):
        # Append product names
        if i < len(names) and names[i].text:
            product_name.append(names[i].text.strip())
        else:
            product_name.append('N/A')
        
        # Append prices
        if i < len(prices) and prices[i].text:
            prices_list.append(prices[i].text.strip())
        else:
            prices_list.append('N/A')
        
        # Append descriptions
        if i < len(descriptions) and descriptions[i].text:
            descriptions_list.append(descriptions[i].text.strip())
        else:
            descriptions_list.append('N/A')
        
        # Append reviews
        if i < len(reviews) and reviews[i].text:
            reviews_list.append(reviews[i].text.strip())
        else:
            reviews_list.append('N/A')

    print(f"Page {items} processed successfully.")

# Create DataFrame
df = pd.DataFrame({
    'Products': product_name,
    'Prices': prices_list,
    'Reviews': reviews_list,
    'Description': descriptions_list
})

print(df.head())

Page 2 processed successfully.
Page 3 processed successfully.
Page 4 processed successfully.
Page 5 processed successfully.
Page 6 processed successfully.
Page 7 processed successfully.
Page 8 processed successfully.
Failed to fetch page 9. Status code: 429
                                            Products   Prices Reviews  \
0  Samsung Galaxy S21 FE 5G with Snapdragon 888 (...  ₹45,999     4.3   
1                       Apple iPhone 12 (Red, 64 GB)  ₹48,900     4.6   
2                     Apple iPhone 12 (White, 64 GB)  ₹48,900     4.6   
3     SAMSUNG Galaxy S21 FE 5G (Olive Green, 128 GB)  ₹39,999     4.3   
4  Samsung Galaxy S21 FE 5G with Snapdragon 888 (...  ₹45,999     4.3   

                                         Description  
0  8 GB RAM | 128 GB ROM16.26 cm (6.4 inch) Full ...  
1  64 GB ROM15.49 cm (6.1 inch) Super Retina XDR ...  
2  64 GB ROM15.49 cm (6.1 inch) Super Retina XDR ...  
3  8 GB RAM | 128 GB ROM16.26 cm (6.4 inch) Full ...  
4  8 GB RAM | 128 GB ROM16.2

## By the end of the loop:

All four lists (product_name, prices_list, descriptions_list, reviews_list) will have the same length (max_items) for the current page.

Missing or unavailable data will be filled with 'N/A', avoiding errors when creating the DataFrame.

In [23]:
df.head()

,Products,Prices,Reviews,Description
0,Samsung Galaxy S21 FE 5G with Snapdragon 888 (...,"₹45,999",4.3,8 GB RAM | 128 GB ROM16.26 cm (6.4 inch) Full ...
1,"Apple iPhone 12 (Red, 64 GB)","₹48,900",4.6,64 GB ROM15.49 cm (6.1 inch) Super Retina XDR ...
2,"Apple iPhone 12 (White, 64 GB)","₹48,900",4.6,64 GB ROM15.49 cm (6.1 inch) Super Retina XDR ...
3,"SAMSUNG Galaxy S21 FE 5G (Olive Green, 128 GB)","₹39,999",4.3,8 GB RAM | 128 GB ROM16.26 cm (6.4 inch) Full ...
4,Samsung Galaxy S21 FE 5G with Snapdragon 888 (...,"₹45,999",4.3,8 GB RAM | 128 GB ROM16.26 cm (6.4 inch) Full ...


In [24]:
df.shape

(168, 4)

In [25]:
df.isnull().sum()

Products       0
Prices         0
Reviews        0
Description    0
dtype: int64

In [39]:
df.duplicated().sum()

16

In [38]:
# Drop duplicate rows
df2 = df.drop_duplicates()

# Reset the index after dropping duplicates
df2 = df2.reset_index(drop=True)

print("Duplicates removed. DataFrame shape:", df2.shape)

Duplicates removed. DataFrame shape: (152, 4)
